In [1]:
!rm /root/.kaggle
!mkdir /root/.kaggle
!mv /content/kaggle.json /root/.kaggle
!kaggle datasets download -d plameneduardo/sarscov2-ctscan-dataset
!mkdir covid
!unzip /content/sarscov2-ctscan-dataset.zip -d covid

rm: cannot remove '/root/.kaggle': No such file or directory
 97% 223M/230M [00:07<00:00, 19.2MB/s]
100% 230M/230M [00:07<00:00, 31.2MB/s]
Archive:  /content/sarscov2-ctscan-dataset.zip
  inflating: covid/COVID/Covid (1).png  
  inflating: covid/COVID/Covid (10).png  
  inflating: covid/COVID/Covid (100).png  
  inflating: covid/COVID/Covid (1000).png  
  inflating: covid/COVID/Covid (1001).png  
  inflating: covid/COVID/Covid (1002).png  
  inflating: covid/COVID/Covid (1003).png  
  inflating: covid/COVID/Covid (1004).png  
  inflating: covid/COVID/Covid (1005).png  
  inflating: covid/COVID/Covid (1006).png  
  inflating: covid/COVID/Covid (1007).png  
  inflating: covid/COVID/Covid (1008).png  
  inflating: covid/COVID/Covid (1009).png  
  inflating: covid/COVID/Covid (101).png  
  inflating: covid/COVID/Covid (1010).png  
  inflating: covid/COVID/Covid (1011).png  
  inflating: covid/COVID/Covid (1012).png  
  inflating: covid/COVID/Covid (1013).png  
  inflating: covid/COVID/Covi

In [2]:
%tensorflow_version 1.x
from keras import layers
from keras.layers import Flatten, Dense, Input, Activation, Dropout, Conv2D, MaxPooling2D, BatchNormalization
from keras.models import Model
from keras.preprocessing import sequence, image
from keras.optimizers import Adam
from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator
from keras import callbacks
from keras.models import Sequential

TensorFlow 1.x selected.


Using TensorFlow backend.


In [3]:
IMAGE_SIZE = 200

batch_size=16

In [4]:
datagen = ImageDataGenerator(rescale=1./255,
                                             horizontal_flip=True,
                                             rotation_range=45,
                                             zoom_range=0.2, 
                                             width_shift_range=0.1,
                                             height_shift_range=0.1,
                                             shear_range=0.05,
                                             channel_shift_range=0.1,
                                             validation_split=0.2
                             )

In [5]:
train_generator = datagen.flow_from_directory(
        '/content/covid',  # this is the target directory
        color_mode="rgb",
        target_size=(200,200),  # all images will be resized to 150x150
        batch_size=batch_size,
        shuffle=True,
        class_mode='binary',
        seed=45,
        classes=['COVID','non-COVID'],
	      subset="training"       
       )

Found 1986 images belonging to 2 classes.


In [6]:
validation_generator = datagen.flow_from_directory(
        '/content/covid',  # this is the target directory
        color_mode="rgb",
        target_size=(200,200),  # all images will be resized to 150x150
        batch_size=batch_size,
        shuffle=False,
        class_mode='binary',
        seed=45,
        subset="validation"       
       )

Found 495 images belonging to 2 classes.


In [7]:
!pip install keras_efficientnets
from keras_efficientnets import EfficientNetB5
model = EfficientNetB5(include_top=False, weights='imagenet',pooling='avg',input_shape=(200,200,3))
x = model.output


x = Dense(2048)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)          
x = Dropout(0.25)(x)


x = Dense(32)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)          
x = Dropout(0.25)(x)


x = Dense(16)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)          
x = Dropout(0.25)(x)


predictions = Dense(1, activation='sigmoid')(x)

model= Model(input = model.input, output = predictions)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
115302400/115295288 [==============================] - 5s 0us/step


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:27: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


In [8]:
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 200, 200, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 100, 100, 48) 1296        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 100, 100, 48) 192         conv2d_1[0][0]                   
__________________________________________________________________________________________________
swish_1 (Swish)                 (None, 100, 100, 48) 0           batch_normalization_1[0][0]      
____________________________________________________________________________________________

In [9]:
filepath="efficient_model.hdf5" # checkpoint
checkpoint = callbacks.ModelCheckpoint(filepath, monitor='val_binary_accuracy',save_weights_only=True, verbose=1, save_best_only=True, mode='max')
adam = Adam()

In [10]:
model.compile(
          optimizer=adam,
          loss='binary_crossentropy',
          metrics=['binary_accuracy'])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [11]:
final_model=model.fit_generator(train_generator,
                    steps_per_epoch=train_generator.samples //batch_size ,
                    validation_data = validation_generator,
                    validation_steps=validation_generator.samples // batch_size,
                    epochs=16,
                    verbose=1,
                    shuffle=True,
                    callbacks=[checkpoint],
                    # max_queue_size=25, 
                    # # workers=2,
                    # # use_multiprocessing=True
                    )


Epoch 1/16
124/124 [==============================] - 186s 1s/step - loss: 0.4350 - binary_accuracy: 0.8107 - val_loss: 0.0141 - val_binary_accuracy: 0.6292

Epoch 00001: val_binary_accuracy improved from -inf to 0.62917, saving model to efficient_model.hdf5
Epoch 2/16
124/124 [==============================] - 138s 1s/step - loss: 0.3113 - binary_accuracy: 0.8929 - val_loss: 0.0300 - val_binary_accuracy: 0.7203

Epoch 00002: val_binary_accuracy improved from 0.62917 to 0.72025, saving model to efficient_model.hdf5
Epoch 3/16
124/124 [==============================] - 138s 1s/step - loss: 0.2897 - binary_accuracy: 0.8929 - val_loss: 0.0203 - val_binary_accuracy: 0.6555

Epoch 00003: val_binary_accuracy did not improve from 0.72025
Epoch 4/16
124/124 [==============================] - 138s 1s/step - loss: 0.2405 - binary_accuracy: 0.9117 - val_loss: 0.0276 - val_binary_accuracy: 0.6639

Epoch 00004: val_binary_accuracy did not improve from 0.72025
Epoch 5/16
124/124 [==================